# Programme de Matching

Regarder si l'établissement est fermé ou pas

## Moteur de recherche TEST

* Insee
  * http://avis-situation-sirene.insee.fr/IdentificationListeSiret.action
* INPI/TC
  * https://data.inpi.fr/
* Infogreffe
  * https://www.infogreffe.fr/

## Preparation fichier

### Variable Ville

- process:
    - creer variables avec numeric seulement
    - recreer ville 2 si test pas NAN pour avoir l'arrondissement
    - virer les differentes informations dans ville via regex
    
### Step 1: Match uniquement les 1 dans INSEE/INPI

Les siren qui ont uniquement un seul établissement a l'INSEE et à l'INPI.

### Step 2: match Ville/CP/CC

### Step 3: Match adresse

## Description Algorithm

## Description Input

## Description Output

In [1]:
from inpi_insee import siretisation

%load_ext autoreload
%autoreload 2

C:\Users\PERNETTH\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
param = {
    'insee': r'data\input\insee_2017_7627977.gz',
    'inpi_etb': r'data\input\inpi_etb_stock_7552898.gz',
}
# 4824158 SIREN a trouver!
al_siret = siretisation.siretisation_inpi(param)

In [ ]:
from itertools import compress, product

def combinations(items):
    return ( set(compress(items,mask)) for 
            mask in product(*[[0,1]]*len(items)))

all_list = ['ncc', 'Code_Postal', 'Code_Commune']

### Match 1-1

### Merge ville/code postale/ code commune

In [3]:
%%time
inpi_col = ['siren',
            'index',
            'Type',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adresse_new_clean_reg',
           'INSEE',
           'Date_Début_Activité',
           'digit_inpi']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adresse_new_clean_reg': 'object',
    'INSEE':'object',
    'Date_Début_Activité':'object',
    'digit_inpi':'object'
}

Wall time: 0 ns



Variable INSEE -> etablissementSiege
Variable INPI -> Type

- PRI    3004262 -> False
- SIE    2362691 -> True
- SEP    1670220 -> True
- SEC     515725 -> False

In [4]:
df_ets = r'data\input\inpi_etb_stock_7552898.gz'
inpi = al_siret.import_dask(file=df_ets,
                            usecols=inpi_col,
                            dtype=inpi_dtype,
                            parse_dates = False)

In [5]:
df_no_duplication, df_duplication = al_siret.create_test(
    df_input=inpi,
    left_on=['siren','ncc',
             'Code_Postal','Code_Commune',
             'INSEE','digit_inpi'],
    right_on=['siren','libelleCommuneEtablissement',
              'codePostalEtablissement', 'codeCommuneEtablissement',
             'typeVoieEtablissement','numeroVoieEtablissement'])

[########################################] | 100% Completed |  2min 11.7s


In [6]:
df_no_duplication.groupby('origin')['origin'].count()

origin
test_1_no_duplication    4312255
test_2_no_duplication      24910
test_3_no_duplication       3184
Name: origin, dtype: int64

In [7]:
df_duplication.groupby('origin')['origin'].count()

origin
test_2_duplication    16580
test_3_duplication    10732
Name: origin, dtype: int64

On regarde le nombre de doublons -> meme variables de matching mais plusieurs possibilité a l'INSEE:

- Plusieurs siret pour un meme endroit

Process temporaire -> on sauvegarde les fichiers avec indication des tests:

- Après le merge full
    - Test 1: doublon
        - non: Save-> test1_nodup_{size}
        - oui:
            - Test 2: Date equal
                - oui:
                    - Test 2 bis: doublon
                        - non: Save-> test2_nodup_{size}
                        - oui: Save-> test2_dup_{size}
                - non:
                    - Test 3: Date sup
                        - oui:
                            - Test 2 bis: doublon
                                - non: Save-> test3_nodup_{size}
                                - oui: Save-> test3_dup_{size}
                        - non: Save-> test3_specialtreat_{size}
                            

In [ ]:
## test
doublou_sur['index'].nunique()
doublou_passur['index'].nunique()
doublon.shape[0] - ( doublou_sur.shape[0] + doublou_passur.shape[0])

### Test sur pas doublon

On applique 5 tests sur les champs des possibles afin d'avoir une meilleure visibilité 

- Test 1: address libelle
    - Si mots dans inpi est contenu dans INSEE, True
- Test 1 bis: address complement
    - Si mots dans inpi est contenu dans INSEE, True
- Test 2: Date
    - dateCreationEtablissement >= Date_Début_Activité OR Date_Début_Activité = NaN OR (nombre SIREN a l'INSEE = 1 AND nombre SIREN des variables de matching = 1), True
- Test 3: siege
    - Type = ['SEP', 'SIE'] AND siege = true, True
- Test 4: voie
    - Type voie INPI = Type voie INSEE, True
- Test 5: numero voie
    - Numero voie INPI = Numero voie INSEE, True

In [23]:
df_no_duplication.shape

(4340349, 34)

In [24]:
pure_match = al_siret.assess_test(df = df_no_duplication)

[########################################] | 100% Completed |  5min 30.3s
[########################################] | 100% Completed |  5min 30.4s
[########################################] | 100% Completed |  1.4s
[########################################] | 100% Completed |  1.5s


In [34]:
pure_match.to_csv(r'data\output\pure_match_{}.gz'.format(pure_match.shape[0]),
                  compression = 'gzip')

In [33]:
pure_match['count_duplicates_final'].value_counts()

1    4340349
Name: count_duplicates_final, dtype: int64

In [ ]:
import numpy as np
import pandas as pd

## Doublons

In [183]:
al_siret.test_doublon(df_duplication)

[########################################] | 100% Completed |  2.1s
[########################################] | 100% Completed |  2.1s
[########################################] | 100% Completed |  2.2s
[########################################] | 100% Completed |  2.2s
[########################################] | 100% Completed |  2.3s
[########################################] | 100% Completed |  2.3s
[########################################] | 100% Completed |  2.4s
[########################################] | 100% Completed |  2.4s
[########################################] | 100% Completed |  2.5s
[########################################] | 100% Completed |  2.6s
[########################################] | 100% Completed |  2.7s
[########################################] | 100% Completed |  2.7s
[########################################] | 100% Completed |  2.1s
[########################################] | 100% Completed |  2.2s
[########################################] | 100

In [36]:
df_duplication.shape

(27312, 34)

In [35]:
df_duplication['index'].nunique()

11168

In [168]:
duplicates_ = al_siret.assess_test(df = df_duplication)

[########################################] | 100% Completed |  1.9s
[########################################] | 100% Completed |  2.0s
[########################################] | 100% Completed |  2.1s
[########################################] | 100% Completed |  2.2s
[########################################] | 100% Completed |  2.2s
[########################################] | 100% Completed |  2.2s
[########################################] | 100% Completed |  2.3s
[########################################] | 100% Completed |  2.4s
[########################################] | 100% Completed |  2.1s
[########################################] | 100% Completed |  2.2s
[########################################] | 100% Completed |  2.3s
[########################################] | 100% Completed |  2.4s
[########################################] | 100% Completed |  2.5s
[########################################] | 100% Completed |  2.6s
[########################################] | 100

In [169]:
duplicates_.shape

(27312, 44)

In [170]:
duplicates_['index'].nunique()

11168

In [171]:
df_not_duplicate = pd.DataFrame()
copy_duplicate = duplicates_.copy()

for i in ['test_join_address','test_address_libelle', 'test_address_complement']:
    ### split duplication
    test_1 = al_siret.split_duplication(
        copy_duplicate[
            copy_duplicate[i].isin([True])]
    )
    
    ### append unique
    df_not_duplicate = (
        df_not_duplicate
        .append(test_1['not_duplication']
                .assign(test = i)
               )
    )
    
    copy_duplicate = (copy_duplicate
                   .loc[~copy_duplicate['index'].isin(
                       pd.concat([
                           test_1['duplication'],
                           test_1['not_duplication']
                       ], axis = 0)['index']
                       .drop_duplicates())])

In [172]:
df_not_duplicate.to_csv(r'data\output\not_duplicate_{}.gz'.format(df_not_duplicate.shape[0]),
                  compression = 'gzip')

(5045, 45)

In [179]:
# Special treatment
(duplicates_[
    ~duplicates_['index']
    .isin(df_not_duplicate['index'])]
 .to_csv(r'data\output\not_duplicate_{}.gz'.format(df_not_duplicate.shape[0]),
                  compression = 'gzip'))

(15440, 44)

# Archive

In [ ]:
df_ets = r'data\input\inpi_etb_stock_7552898.gz'
inpi = al_siret.import_dask(file=df_ets,
                       usecols=inpi_col, dtype=inpi_dtype)

df_nomatch = pd.DataFrame()
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(df_input=df_input,
                                  option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

    
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='1_ville_cp_cc',
        to_csv=True)

### Loop

In [ ]:
%%time
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


#df_nomatch = pd.DataFrame()
df_match = pd.DataFrame()
for chunk in [0,1
              ,2,3,4,5,6,7,8,9, 10,11,12
             ]:
    csv_file = r'data\input\unmatched\chunk\chunk_{}.gz'.format(chunk)
    inpi = al_siret.import_dask(file=csv_file,
                       usecols=inpi_col, dtype=inpi_dtype)
    df_input = inpi
    total_match = pd.DataFrame()
    for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
        df_input_ = al_siret.merge_siren_candidat(df_input=df_input,
                                      option=i,
                                      regex_go = True)

        df_input = df_input_['unmatch']
        total_match = total_match.append(df_input_['true_match']) 
    #df_nomatch = df_nomatch.append(df_input)
    df_match = df_match.append(total_match)

In [ ]:
df_temp = pd.DataFrame()
for i in range(0,13):
    csv_file = r'data\input\unmatched\chunk\chunk_{}.gz'.format(i)
    df_temp = df_temp.append(pd.read_csv(csv_file,
                                         dtype= inpi_dtype)
                            )

In [ ]:
299986 / 4979272

In [ ]:
al_siret.match_unmatch(
        df_inpi_initial=df_temp,
        df_inpi_mergeboth=df_match.drop(columns = '_merge'),
        step='3_adress',
        to_csv=True)

### Matching voie

In [ ]:
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}

In [ ]:
#temp = al_siret.merge_siren_candidat(df_input=inpi,
#                              regex_go=True, 
#                              matching_voie=True,
#                              option=['ncc', 'libelleCommuneEtablissement'])

In [ ]:
import pandas as pd
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_3_adress_299986.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

In [ ]:
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='4_voie',
        to_csv=True)

### Relax contrainte Regex

Sans numero de voie

In [ ]:
import pandas as pd
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_4_voie_167801.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=False,
        relax_regex = True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

In [ ]:
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='5_voie_relax',
        to_csv=True)

Avec la voie

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}

inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_5_relax_116575.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=True,
        relax_regex = True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

In [ ]:
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_5_relax_116575.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
check = al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='6_voie_relax',
        to_csv=False)

In [ ]:
check.head()

In [ ]:
check['count_initial_inpi'].value_counts()

### Match etablissement principal ouvert

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_6_voie_relax_112847.gz',
                       usecols=inpi_col, dtype=inpi_dtype)

In [ ]:
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=False,
        relax_regex = True,
        siege_etat=True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])
    
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='7_siege_ouvert',
        to_csv=True)

### Complement d adresse

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_7_siege_ouvert_85312.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=False,
        relax_regex = True,
        siege_etat=True,
        option=i,
        var_adress_insee = 'complementAdresseEtablissement')

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])
    


In [ ]:
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='8_siege_ouvert_complement',
        to_csv=True)

In [ ]:
1- (80474/4979272)

### Test sans regex mais siege_etat

ie si principale-ouvert dans la ville/CP/CC

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_8_siege_ouvert_complement_80474.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=False,
        matching_voie=False,
        relax_regex = False,
        siege_etat=True,
        option=i,
        var_adress_insee = 'libelleVoieEtablissement')

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

In [ ]:
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='9_siege_ouvert_no_regex',
        to_csv=True)

In [ ]:
round(1 - (68131 / 4979272),3)

# Temp

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


inpi = al_siret.import_dask(file=r'data\output\match_9_siege_ouvert_no_regex_12343.gz',
                       usecols=inpi_col, dtype=inpi_dtype)

In [ ]:
inpi.head()

In [ ]:
inpi['Adresse_new_clean_reg'].head()

## Amelioration matching

In [ ]:
import pandas as pd
insee_dtype = {
    'siren': 'object',
    'siret': 'object',
    'dateCreationEtablissement': 'object',
    "etablissementSiege": "object",
    "etatAdministratifEtablissement": "object",
    'complementAdresseEtablissement': 'object',
    'numeroVoieEtablissement': 'object',
    'indiceRepetitionEtablissement': 'object',
    'typeVoieEtablissement': 'object',
    'libelleVoieEtablissement': 'object',
    'codePostalEtablissement': 'object',
    'libelleCommuneEtablissement': 'object',
    'libelleCommuneEtrangerEtablissement': 'object',
    'distributionSpecialeEtablissement': 'object',
    'codeCommuneEtablissement': 'object',
    'codeCedexEtablissement': 'object',
    'libelleCedexEtablissement': 'object',
    'codePaysEtrangerEtablissement': 'object',
    'libellePaysEtrangerEtablissement': 'object',
    'count_initial_insee': 'int'
}

insee_col = ['siren',
             'siret',
             'dateCreationEtablissement',
             "etablissementSiege",
             "etatAdministratifEtablissement",
             'complementAdresseEtablissement',
             'numeroVoieEtablissement',
             'indiceRepetitionEtablissement',
             'typeVoieEtablissement',
             'libelleVoieEtablissement',
             'codePostalEtablissement',
             'libelleCommuneEtablissement',
             'libelleCommuneEtrangerEtablissement',
             'distributionSpecialeEtablissement',
             'codeCommuneEtablissement',
             'codeCedexEtablissement',
             'libelleCedexEtablissement',
             'codePaysEtrangerEtablissement',
             'libellePaysEtrangerEtablissement',
             'count_initial_insee']
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}

insee = al_siret.import_dask(file=r'data\input\insee_2017_7480120.gz',
                             usecols=insee_col, dtype=insee_dtype)


In [ ]:
list(insee)

In [ ]:
#insee['etatAdministratifEtablissement'].value_counts().compute()
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_8_siege_ouvert_complement_80474.gz',
                       usecols=inpi_col, dtype=inpi_dtype)

In [ ]:
inpi.head()

In [ ]:
insee[
    (insee['siren'].isin(['515226462']))
#& (insee['libelleVoieEtablissement'].isin(['JACQUARD']))
].compute()

In [ ]:
inpi[inpi['siren'].isin(['515226462'])].compute()

In [ ]:
inpi.loc[inpi['count_initial_inpi'].isin([1])].head(10)

In [ ]:
inpi.loc[inpi['siren'].isin(['322385949'])].compute()

In [ ]:
inpi['Type'].value_counts().compute()

In [ ]:
insee[insee['siren'].isin(['322385949'])].compute()

Verification conservative:

- A l'INSEE, si etatAdministratifEtablissement == A & etablissementSiege == True, alors on fait le sous ensemle
- Verifier avec les dates de création de l'établissement

In [ ]:
insee.loc[
            (insee['etablissementSiege'].isin(['true'])) 
            & (insee['etatAdministratifEtablissement'].isin(['A']))
            ].compute() 

In [ ]:
1 - (112847 / 4979272)

In [ ]:
amelioration possible:
    
- 381980788 -> Zone Artisanale du Creusât ->transformer en ZA